In [1]:
# Qpt

In [1]:
import pickle
import time
from typing import List
import datetime as dt
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

import numpy.testing as npt

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.data_analysis.projected_gradient_descent_backtracking import ProjectedGradientDescentBacktracking, ProjectedGradientDescentBacktrackingOption
from quara.data_analysis.weighted_probability_based_squared_error import WeightedProbabilityBasedSquaredError, WeightedProbabilityBasedSquaredErrorOption
from quara.data_analysis.weighted_relative_entropy import WeightedRelativeEntropy, WeightedRelativeEntropyOption
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.state import (
    State,
    get_x0_1q,
    get_x1_1q,
    get_y0_1q,
    get_y1_1q,
    get_z0_1q,
    get_z1_1q,
)
from quara.objects.povm import (
    Povm,get_x_povm,
    get_y_povm,
    get_z_povm
)
from quara.objects.gate import (
    Gate, get_depolarizing_channel,
    get_x_rotation,
    get_amplitutde_damping_channel
)
from quara.objects.qoperation import QOperation
from quara.protocol.qtomography.standard.standard_qpt import StandardQpt
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.loss_minimization_estimator import LossMinimizationEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)

from quara.data_analysis.simulation import StandardQTomographySimulationSetting

from quara.settings import Settings
Settings.get_atol()

1e-13

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

# Tester Objects (State)
# |+><+|
state_x0 = get_x0_1q(c_sys)
# |+i><+i|
state_y0 = get_y0_1q(c_sys)
# |0><0|
state_z0 = get_z0_1q(c_sys)
# |1><1|
state_z1 = get_z1_1q(c_sys)
tester_states = [state_x0, state_y0, state_z0, state_z1]

In [4]:
tester_povms = [get_x_povm(c_sys), get_y_povm(c_sys), get_z_povm(c_sys)]

In [5]:
# True Object
true_objects = []
#true_objects.append(get_depolarizing_channel(p=0, c_sys=c_sys))
true_objects.append(get_depolarizing_channel(p=0.05, c_sys=c_sys))
#true_objects.append(get_depolarizing_channel(p=1, c_sys=c_sys))
#true_objects.append(get_x_rotation(theta=np.pi/2, c_sys=c_sys))
#true_objects.append(get_x_rotation(theta=np.pi, c_sys=c_sys))
#true_objects.append(get_amplitutde_damping_channel(gamma=0.1, c_sys=c_sys))

true_object = true_objects[0]

In [32]:
num_data = [100, 1000]
n_rep = 10

case_name_list = [
    "LinearEstimator(True)",
    "LinearEstimator(False)",
    "ProjectedLinearEstimator(True)",
    "ProjectedLinearEstimator(False)",
    "Maximum-Likelihood(True)",
    "Maximum-Likelihood(False)",
    "Least Squares(True)",
    "Least Squares(False)",
]

seed = 777
qtomography_list = [
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=True, seed=seed),
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=False, seed=seed),
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=True, seed=seed),
    StandardQpt(tester_states, tester_povms, on_para_eq_constraint=False, seed=seed),
    StandardQpt(
        tester_states,
        tester_povms,
        on_para_eq_constraint=True,
        eps_proj_physical=1e-13,
        seed=seed,
    ),
    StandardQpt(
        tester_states,
        tester_povms,
        on_para_eq_constraint=False,
        eps_proj_physical=1e-13,
        seed=seed,
    ),
    StandardQpt(
        tester_states,
        tester_povms,
        on_para_eq_constraint=True,
        eps_proj_physical=1e-13,
        seed=seed,
    ),
    StandardQpt(
        tester_states,
        tester_povms,
        on_para_eq_constraint=False,
        eps_proj_physical=1e-13,
        seed=seed,
    ),
]

para_list = [
    True,
    False,
    True,
    False,
    True,
    False,
    True,
    False,
]

order = "eq_ineq"

estimator_list = [
    LinearEstimator(),
    LinearEstimator(),
    ProjectedLinearEstimator(mode_proj_order=order),
    ProjectedLinearEstimator(mode_proj_order=order),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
    LossMinimizationEstimator(),
]

loss_list = [
    None,
    None,
    None,
    None,
    WeightedRelativeEntropy(12),
    WeightedRelativeEntropy(16),
    WeightedProbabilityBasedSquaredError(12),
    WeightedProbabilityBasedSquaredError(16),
]

loss_option_list = [
    None,
    None,
    None,
    None,
    WeightedRelativeEntropyOption("identity"),
    WeightedRelativeEntropyOption("identity"),
    WeightedProbabilityBasedSquaredErrorOption("identity"),
    WeightedProbabilityBasedSquaredErrorOption("identity"),
]

algo_list = [
    None,
    None,
    None,
    None,
    ProjectedGradientDescentBacktracking(),
    ProjectedGradientDescentBacktracking(),
    ProjectedGradientDescentBacktracking(),
    ProjectedGradientDescentBacktracking(),
]

algo_option_list = [
    None,
    None,
    None,
    None,
    ProjectedGradientDescentBacktrackingOption(
        mode_stopping_criterion_gradient_descent="sum_absolute_difference_variable",
        num_history_stopping_criterion_gradient_descent=1,
        mode_proj_order=order
    ),
    ProjectedGradientDescentBacktrackingOption(
        mode_stopping_criterion_gradient_descent="sum_absolute_difference_variable",
        num_history_stopping_criterion_gradient_descent=1,
        mode_proj_order=order
    ),
    ProjectedGradientDescentBacktrackingOption(
        mode_stopping_criterion_gradient_descent="sum_absolute_difference_variable",
        num_history_stopping_criterion_gradient_descent=1,
        mode_proj_order=order
    ),
    ProjectedGradientDescentBacktrackingOption(
        mode_stopping_criterion_gradient_descent="sum_absolute_difference_variable",
        num_history_stopping_criterion_gradient_descent=1,
    mode_proj_order=order
    ),
]

estimation_results_list = []
elapsed_times = []
simulation_settings = []

for i, name in enumerate(case_name_list):
    qtomography = qtomography_list[i]
    estimator = estimator_list[i]
    loss = loss_list[i]
    loss_option = loss_option_list[i]
    algo = algo_list[i]
    algo_option = algo_option_list[i]

    start = time.time()
    print(f"Case {i}: {name}")
    print(f"Parametorization: {para_list[i]}")
    print(f"Type of qtomography: {qtomography.__class__.__name__}")
    print(f"Estimator: {estimator.__class__.__name__}")

    estimation_results = data_analysis.estimate(
        qtomography=qtomography,
        true_object=true_object,
        num_data=num_data,
        estimator=estimator,
        loss=loss,
        loss_option=loss_option,
        algo=algo,
        algo_option=algo_option,
        iteration=n_rep,
    )
    estimation_results_list.append(estimation_results)

    # stock settings of this simulation
    simulation_setting = StandardQTomographySimulationSetting(
        name=name,
        estimator=estimator,
        loss=loss,
        loss_option=loss_option,
        algo=algo,
        algo_option=algo_option,
        true_object=true_object,
        tester_objects=tester_states + tester_povms,
        n_rep=n_rep,
        seed=seed
    )
    simulation_settings.append(simulation_setting)

    elapsed_time = time.time() - start
    print("elapsed_time:{0}".format(elapsed_time / 60) + "[min]\n")
    elapsed_times.append(elapsed_time)

 10%|█         | 1/10 [00:00<00:01,  8.50it/s]

Case 0: LinearEstimator(True)
Parametorization: True
Type of qtomography: StandardQpt
Estimator: LinearEstimator


 20%|██        | 2/10 [00:00<00:00, 14.34it/s]

elapsed_time:0.012786213556925457[min]

Case 1: LinearEstimator(False)
Parametorization: False
Type of qtomography: StandardQpt
Estimator: LinearEstimator


 10%|█         | 1/10 [00:00<00:01,  6.08it/s]

...

elapsed_time:9.102739016215006[min]

Case 6: Least Squares(True)
Parametorization: True
Type of qtomography: StandardQpt
Estimator: LossMinimizationEstimator


  0%|          | 0/10 [00:00<?, ?it/s]

elapsed_time:1.078106955687205[min]

Case 7: Least Squares(False)
Parametorization: False
Type of qtomography: StandardQpt
Estimator: LossMinimizationEstimator


100%|██████████| 10/10 [01:47<00:00, 10.77s/it]

elapsed_time:1.795636252562205[min]



In [51]:
report.export_report(
    f"qpt_nrep={n_rep}_order={order}.pdf",
    estimation_results_list=estimation_results_list,
    simulation_settings=simulation_settings,
    seed=seed,
)

​Generating table of computation time ...
​Generating table of tolerance of physicality constraint violation ...
​Generating table of experimental conditions ...
Generating case list ...
Computation time of estimators ...


100%|██████████| 10/10 [00:00<00:00, 12690.78it/s]

​​Generating MSE of empirical distributions blocks ...


​​Generating consictency test blocks ...
​Generating a graph for MSE ...
  LinearEstimator(True)=[0.08906000000000011, 0.007834800000000013]
  LinearEstimator(False)=[0.08938, 0.007004800000000003]
  ProjectedLinearEstimator(True)=[0.051741886010387136, 0.00603727872160328]
  ProjectedLinearEstimator(False)=[0.052025302270329976, 0.00697164225382773]
  Maximum-Likelihood(True)=[0.0274356568505498, 0.004678587007470968]
  Maximum-Likelihood(False)=[0.023400168869827355, 0.004357857278584862]
  Least Squares(True)=[0.04341417722579008, 0.003956543515917558]
  Least Squares(False)=[0.03491375488402609, 0.0045475583147375695]
  LinearEstimator(True)=[0.08906000000000011, 0.007834800000000013]
  ProjectedLinearEstimator(True)=[0.051741886010387136, 0.00603727872160328]
  Maximum-Likelihood(True)=[0.0274356568505498, 0.004678587007470968]
  Least Squares(True)=[0.04341417722579008, 0.003956543515917558]
  LinearEstimator(False)=[0.08938, 0.007004800000000003]
  ProjectedLinearEstimator(False

In [36]:
from quara.data_analysis.simulation_check import StandardQTomographySimulationCheck

results = []
for i in range(len(simulation_settings)):
    sim_check = StandardQTomographySimulationCheck(simulation_settings[i], estimation_results_list[i])
    print(f"Case {i}: {simulation_settings[i].name}")
    result = sim_check.execute_physicality_violation_check()
    results.append(result)
    print("===============================")
    print()

Case 0: LinearEstimator(True)
[OK] N=100 is_eq_constraint_satisfied_all
True=10, False=0, eps=1e-13
[OK] N=1000 is_eq_constraint_satisfied_all
True=10, False=0, eps=1e-13

Case 1: LinearEstimator(False)
[Skipped] Physicality Violation Check 
If on_para_eq_constraint is False in Linear Estimator, nothing is checked.

Case 2: ProjectedLinearEstimator(True)
[OK] N=100 physicality violation check
True=10, False=0
[OK] N=1000 physicality violation check
True=10, False=0

Case 3: ProjectedLinearEstimator(False)
[OK] N=100 physicality violation check
True=10, False=0
[OK] N=1000 physicality violation check
True=10, False=0

Case 4: Maximum-Likelihood(True)
[OK] N=100 is_eq_constraint_satisfied_all
True=10, False=0, eps=1e-13
[OK] N=1000 is_eq_constraint_satisfied_all
True=10, False=0, eps=1e-13
[OK] N=100 is_ineq_constraint_satisfied_all
True=10, False=0, eps=1e-05
[OK] N=1000 is_ineq_constraint_satisfied_all
True=10, False=0, eps=1e-05

Case 5: Maximum-Likelihood(False)
[OK] N=100 is_eq_cons

In [16]:
physicality_violation_check.get_ineq_const_eps()

1e-05

In [47]:
from quara.data_analysis.simulation_check import StandardQTomographySimulationCheck

results = []
for i in range(len(simulation_settings)):
    sim_check = StandardQTomographySimulationCheck(simulation_settings[i], estimation_results_list[i])
    print(f"Case {i}: {simulation_settings[i].name}")
    result = sim_check.execute_all()
    results.append(result)
    print("===============================")
    print()

100%|██████████| 10/10 [00:00<00:00, 16797.37it/s]

Case 0: LinearEstimator(True)



100%|██████████| 10/10 [00:00<00:00, 19608.71it/s]

[OK] N=100
|MSE_EmpiDist - MSE_Analytical| = 0.0011699999999999974
eps = 0.038971989941495186
[OK] N=1000
|MSE_EmpiDist - MSE_Analytical| = 0.00019779999999999104
eps = 0.00543930010203518
[OK] Consistency Check
result=0.0
eps=1e-10
result < eps: True
[OK] N=100
|MSE_Linear - MSE_Analytical| = 0.0051600000000000396
eps = 0.10134842870020236
[OK] N=1000
|MSE_Linear - MSE_Analytical| = 0.0005551999999999935
eps = 0.010783522318797319
[OK] N=100 is_eq_constraint_satisfied_all
True=10, False=0, eps=1e-13
[OK] N=1000 is_eq_constraint_satisfied_all
True=10, False=0, eps=1e-13
========== Summary ============
Name: LinearEstimator(True)
MSE of Empirical Distributions: OK
Consistency: OK (to_be_checked=False)
MSE of estimators: OK
Physicality Violation: OK


Case 1: LinearEstimator(False)



  0%|          | 0/10 [00:00<?, ?it/s]

...

100%|██████████| 10/10 [00:00<00:00, 5684.88it/s]


[OK] N=100
MSE_Linear - MSE_LossMinimizationEstimator = 0.05383983113017266
[OK] N=1000
MSE_Linear - MSE_LossMinimizationEstimator = 0.002784542721415135
[OK] N=100 is_eq_constraint_satisfied_all
True=10, False=0, eps=1e-05
[OK] N=1000 is_eq_constraint_satisfied_all
True=10, False=0, eps=1e-05
[OK] N=100 is_ineq_constraint_satisfied_all
True=10, False=0, eps=1e-05
[OK] N=1000 is_ineq_constraint_satisfied_all
True=10, False=0, eps=1e-05
========== Summary ============
Name: Maximum-Likelihood(False)
MSE of Empirical Distributions: OK
Consistency: OK (to_be_checked=False)
MSE of estimators: OK
Physicality Violation: OK


Case 6: Least Squares(True)
[OK] N=100
|MSE_EmpiDist - MSE_Analytical| = 0.001030000000000024
eps = 0.042845956635369935
[OK] N=1000
|MSE_EmpiDist - MSE_Analytical| = 0.0011380000000000036
eps = 0.004812476493449094


/Users/tomoko/project/rcast/workspace/quara/quara/data_analysis/simulation_check.py:144: UserWarning:

Estimator MSE is not checked except for LinearEstimator, ProjectedLinearEstimator, Maximum-likelihood.

100%|██████████| 10/10 [00:00<00:00, 5143.86it/s]

[OK] Consistency Check
result=4.221029303602075e-28
eps=1e-10
result < eps: True
[OK] N=100 is_eq_constraint_satisfied_all
True=10, False=0, eps=1e-13
[OK] N=1000 is_eq_constraint_satisfied_all
True=10, False=0, eps=1e-13
[OK] N=100 is_ineq_constraint_satisfied_all
True=10, False=0, eps=1e-05
[OK] N=1000 is_ineq_constraint_satisfied_all
True=10, False=0, eps=1e-05
========== Summary ============
Name: Least Squares(True)
MSE of Empirical Distributions: OK
Consistency: OK (to_be_checked=False)
MSE of estimators: OK
Physicality Violation: OK


Case 7: Least Squares(False)
[OK] N=100
|MSE_EmpiDist - MSE_Analytical| = 0.00291000000000001
eps = 0.043244236610212004
[OK] N=1000
|MSE_EmpiDist - MSE_Analytical| = 0.00040039999999999434
eps = 0.004599422615937793


[OK] Consistency Check
result=1.5735738086587649e-27
eps=1e-10
result < eps: True
[OK] N=100 is_eq_constraint_satisfied_all
True=10, False=0, eps=1e-05
[OK] N=1000 is_eq_constraint_satisfied_all
True=10, False=0, eps=1e-05
[OK] N=100 is_ineq_constraint_satisfied_all
True=10, False=0, eps=1e-05
[OK] N=1000 is_ineq_constraint_satisfied_all
True=10, False=0, eps=1e-05
========== Summary ============
Name: Least Squares(False)
MSE of Empirical Distributions: OK
Consistency: OK (to_be_checked=False)
MSE of estimators: OK
Physicality Violation: OK


